# 要約 
このJupyter Notebookは、Kaggleのコンペティション「LLM 20 Questions」において、言語モデルを使用した「20の質問」ゲームのエージェントを開発することを目的としています。ユーザーは、質問を通じてターゲットワードを絞り込み、できるだけ少ないラウンドでその単語を推測することが求められています。

主な手法として、Pythonのデータ処理ライブラリである`pandas`や数値計算ライブラリの`numpy`、さらに正規表現操作のための`re`、ランダム数生成のための`random`が利用されています。これらのライブラリは、データの管理や解析を容易にし、エージェントの機能を強化します。

このNotebookでは、特に次のような機能が実装されています：
- **単語リストの作成**: さまざまなオブジェクトや地名を含む大規模な単語リストが`allwords`変数に携わっています。
- **質問・推測に基づく応答**: `agent_alpha`と`agent_beta`という2つの主要なエージェントが定義され、質問形式のターンを識別し、それに基づいて応答を生成します。`switchboard`関数を通じて、ターンのタイプに応じた各エージェントの処理フローが管理されています。
- **ランダム化された応答**: `agent_beta`では、確率的に選ばれるカジュアルな応答が含まれており、ゲームにおける戦略的要素を取り入れています。
- **Kaggle環境の構築**: `kaggle_environments`ライブラリを使用して、エージェントの動作をシミュレートし、ゲームを実行するための環境を設定しています。

全体として、このNotebookは、構造化されたコーディングスタイルで、複雑なゲームシステムをシンプルに実現するためのフレームワークを提供しており、言語モデルを基にした対話型の推測エージェントを構築しています。

---


# 用語概説 
以下に、機械学習・深層学習の初心者がつまずきそうな専門用語を簡単に解説します。このノートブック特有のコンテキストや概念に焦点を当てており、一般的によく知られている用語は含めていません。

### 専門用語の解説

1. **エージェント (Agent)**:
   - 問題解決を行うために設計されたプログラムのこと。ここでは、20の質問ゲームのプレイに参与する「質問者エージェント」と「回答者エージェント」が含まれています。

2. **観察 (Observation)**:
   - エージェントが現在の状態や対戦相手から得る情報。例えば、質問や回答の履歴、ターンの種類などが含まれます。

3. **設定 (Configuration)**:
   - エージェントが動作するための環境構成やパラメータ設定。例えば、最大ステップ数やタイムアウトなどの設定が含まれます。

4. **スイッチボード (Switchboard)**:
   - エージェントの処理の流れを管理する機能。どのエージェントが次に行動するかを決定する役割を持っています。

5. **推測 (Guess)**:
   - 質問に基づいてエージェントがターゲットワードを当てようとする行為。20の質問ゲームの主要な要素の一つです。

6. **正規表現 (Regular Expression)**:
   - 文字列を検索・操作するためのパターン記述法。ここでは、特定の形式で囲まれた文字列を見つけだすために使用されています。

7. **アルファベット順 (Lexicographical Order)**:
   - 文字列の並び順をアルファベットの順番に従って並べること。このコンペティションでの質問の一部として使用されます。

8. **二分探索 (Binary Search)**:
   - ソートされたリストから特定の要素を効率よく探す方法。ここでは、取得した情報を元に候補を絞る際に用いられています。

9. **ターン (Turn)**:
   - ゲームにおける一連のアクションを表します。「ask」「guess」「answer」のように、エージェントの動作がこの概念に基づいています。

10. **ダミーエージェント (Dummy Agent)**:
    - 単純な戦略（固定の質問や回答）で動くエージェント。このノートブックでは、比較対象として使用されており、他のエージェントとの性能比較を行うためのものです。

これらの用語や概念は、特定のコンペティションや問題設定に関連しており、初心者にとって理解が難しい場合があります。それでも、基本的な知識をもっている場合は、これらの解説が役立ちます。

---


In [ ]:
# このPython 3環境には、多くの便利な分析ライブラリがインストールされています
# これはkaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、ここではいくつかの便利なパッケージを読み込む方法を示します

import numpy as np # 線形代数関連のライブラリ
import pandas as pd # データ処理用ライブラリ、CSVファイルの入出力（例: pd.read_csv）

# 入力データファイルは、読み取り専用の"../input/"ディレクトリにあります
# 例えば、このコードを実行すると（クリックするかShift+Enterを押すことで）、入力ディレクトリ内のすべてのファイルがリストされます

import os
for dirname, _, filenames in os.walk('/kaggle/input'):  # /kaggle/input/内のすべてのファイルへのパスを取得
    for filename in filenames:
        print(os.path.join(dirname, filename))  # ディレクトリ名とファイル名を結合して表示

# 最大20GBのデータを現在のディレクトリ（/kaggle/working/）に書き込むことができ、これにより"Save & Run All"を使用してバージョンを作成すると出力として保存されます
# また、一時ファイルを/kaggle/temp/に書き込むこともできますが、現在のセッションの外では保存されません

In [ ]:
import random  # ランダムな数値を生成するためのライブラリをインポート
import re  # 正規表現を使用するためのライブラリをインポート

VERSION = 7  # 使用するバージョン番号を設定する。これはプログラムのバージョン管理に役立つ。

In [ ]:
allwords = ['advertisement', 'agave', 'air compressor', 'air conditioner', 'air filter', 'air vent', 'alarm system',
            'analogy', 'anemone', 'anesthesia', 'apple pie', 'aprons', 'aquarium', 'atmosphere', 'auditorium',
            'backrest', 'bacteria', 'baguette', 'balcony', 'bank', 'barber chair', 'barcode', 'bat', 'bath mat',
            'battery bank', 'beanbag', 'bed frame', 'bike', 'bike path', 'binoculars', 'bird cage', 'bird seed',
            'bleachers', 'blinds', 'board games', 'bobby pins', 'bollards', 'bonding agent', 'bookcase', 'bookends',
            'bottled water', 'bread knife', 'bread pudding', 'break room', 'brewery merchandise', 'briefcase',
            'brochure', 'broken glass', 'brownies', 'bug spray', 'bulb', 'bumper sticker', 'bunsen burner', 'butterfly',
            'cabinet', 'calculator', 'cantaloupe', 'car seat', 'card', 'cardboard box', 'cash register', 'cat bed',
            'cat carrier', 'cauliflower', 'ceiling fan', 'cereal', 'latte', 'champagne flute', 'chandelier',
            'cheesecake', 'chessboard', 'chew toy', 'chocolate cake', 'cinnamon roll', 'rags', 'coat rack',
            'coffee beans', 'coffee grinder', 'coffee grounds', 'coffee makers', 'comic book', 'contact lenses',
            'conveyor belt', 'cooling tower', 'coral reefs', 'cream cheese', 'crochet hook', 'croissant', 'cup holder',
            'cupcake', 'curling iron', 'curtains', 'cutlery', 'cutting board', 'dandelion', 'deciduous tree',
            'dental chair', 'desk chairs', 'desk lamp', 'desktop computer', 'diaper', 'dijon mustard', 'dining table',
            'dish rack', 'dish soap', 'disinfectant', 'diving board', 'dog bed', 'dog crate', 'dog shampoo', 'donuts',
            'drain hose', 'drapes', 'duct tape', 'duffle bags', 'dumbbells', 'dump truck', 'duvet', 'dvd', 'dvd player',
            'dynamite', 'ear protection', 'earl grey tea', 'earplug', 'earth', 'edamame', 'edible flowers',
            'electric scooter', 'electric toothbrush', 'electrical outlet', 'electrical panel', 'electrical tape',
            'elevator', 'elliptical trainers', 'emergency exit sign', 'emergency lights', 'energy drink', 'engravings',
            'escalators', 'eucalyptus', 'excavator', 'exercise mat', 'exhaust fan', 'exit sign', 'extension cord',
            'eye mask', 'face mask', 'facial toner', 'fanny pack', 'fertilizer', 'filing cabinet', 'finger food',
            'fire alarm', 'fire escape ladder', 'fire extinguisher', 'fireplace', 'first aid kit', 'fish tank',
            'fishing pier', 'fitness tracker', 'flashlight', 'floor jacks', 'floor mats', 'foam roller', 'fog machine',
            'food bowl', 'food warmers', 'fortune cookie', 'frappuccino', 'free weights', 'french toast',
            'fridge magnet', 'fried rice', 'fungi', 'furniture', 'furniture polish', 'fuse', 'gadget', 'garage door',
            'garbage bag', 'garbage can', 'garbage disposal', 'garbage truck', 'gas mask', 'generator', 'glass table',
            'glove box', 'glove', 'golf cart', 'gps', 'grain bin', 'granola', 'grape vine', 'grapefruit',
            'graphic novel', 'graphing calculator', 'gravity', 'green beans', 'greeting card', 'guard tower',
            'guitar string', 'gym mats', 'habanero pepper', 'hair clip', 'hair dryer', 'hair gel', 'hairbrush',
            'hairspray', 'hamster wheel', 'hamstring curl machine', 'hand dryer', 'hand sanitizer', 'handbag',
            'handrail', 'hard hat', 'hash browns', 'hay bale', 'hazelnut', 'hdmi cable', 'headlamp', 'headphones',
            'hearing aid', 'heart rate monitors', 'heating element', 'hibiscus', 'high chair', 'highlighter',
            'hiking boots', 'holly', 'honeybee', 'hot dog', 'hot water bottle', 'ice cream maker', 'ice cube tray',
            'ice water', 'iced coffee', 'icicle', 'incense', 'incubator', 'index card', 'inhaler', 'ink cartridge',
            'insulation', 'router', 'interstate', 'iris', 'iron', 'ironing board', 'iron bar', 'irrigation system',
            'iv', 'jewelry box', 'journal', 'joystick', 'jumper cables', 'kale smoothie', 'karaoke machine',
            'key chain', 'khaki pants', 'kiosk', 'kitchen cabinet', 'garbage disposal', 'kitchen table',
            'komodo dragon', 'lab coat', 'lanyard', 'laptop', 'laser beam', 'lathe', 'laundry basket', 'lawnmower',
            'lego', 'library card', 'light switch', 'lightbulb', 'lint roller', 'lint trap', 'litter scooper',
            'lottery ticket', 'luggage', 'magazine rack', 'mailbox', 'marshmallow', 'masks', 'matches', 'measuring cup',
            'measuring tape', 'medical records', 'medication dispenser', 'medication', 'medicine cabinet', 'menorah',
            'metal detector', 'mouse', 'microphone stand', 'microscope', 'microwave', 'milling machine', 'mini fridge',
            'mixing bowl', 'mobile phone', 'modem', 'motorcycle helmet', 'movie poster', 'muffin tin', 'muffin',
            'muffler', 'mural', 'mushroom', 'nachos', 'nail clipper', 'nail polish', 'nail polish remover', 'name tag',
            'nap mat', 'necklace', 'nightstand', 'notebook', 'nozzle', 'oak table', 'ocarina', 'oil filter', 'ointment',
            'olive oil', 'onyx', 'orange peel', 'orange tree', 'orange zest', 'outdoor furniture', 'oven door',
            'oven rack', 'owl', 'oxygen tank', 'pacifier', 'packing peanuts', 'pad', 'pajamas', 'pallet jack',
            'pamphlet', 'pants', 'paper towels', 'paperweights', 'paperwork', 'parachute', 'parallel bars', 'pea soup',
            'pencil sharpener', 'perfume', 'personal protective equipment', 'pesticide', 'phone charger', 'photo album',
            'piano', 'pickaxe', 'piping tips', 'placemat', 'planter box', 'plaque', 'plastic gloves', 'plastic wrap',
            'plate', 'play mat', 'playing cards', 'pliers', 'polyester', 'pool cleaner', 'popcorn', 'popcorn machine',
            'portable speaker', 'pot holder', 'potato', 'power drill', 'power lines', 'power strip', 'press box',
            'pressure gauge', 'projector', 'protein bar', 'protein shakes', 'pry bar', 'public address system',
            'pumpkin pie', 'punching bag', 'push pin', 'puzzle', 'pyramid', 'quesadilla', 'rabbit', 'raccoon',
            'radio scanner', 'radish', 'railroad tracks', 'raindrops', 'rat', 'red velvet cake', 'red wine',
            'refrigerator', 'rehearsal room', 'reindeer', 'relish', 'reptile', 'resin', 'respirator', 'restaurants',
            'rice pudding', 'rivet', 'robot arm', 'rolling pin', 'roots', 'rowing machines', 'rug', 'rv',
            'safety harness', 'salad dressing', 'satellite', 'sauna', 'scones', 'scoreboard', 'scrap metal',
            'scratching posts', 'screwdriver', 'security systems', 'seed packet', 'server rack', 'sewing kit',
            'shin guards', 'shipping label', 'shoe rack', 'shower caddy', 'shower mat', 'sieve', 'silicone',
            'silicone mat', 'silver ore', 'sippy cup', 'sketchbook', 'sleeping bag', 'smartphone', 'smartwatch',
            'smoke detector', 'snacks', 'soap dish', 'soap dispenser', 'soap suds', 'soda', 'sofa', 'solar panel',
            'soldering iron', 'sound system', 'soybeans', 'space probe', 'spatula', 'spice rack', 'spider web',
            'sportswear', 'spotlight', 'sprinkler', 'sprinkler system', 'squirrel', 'stadium', 'stage lights',
            'stain remover', 'stained glass window', 'stair stepper', 'staircase', 'stationary bike', 'steam room',
            'sticker', 'sticky notes', 'storage bin', 'stove', 'street sign', 'sugar packet', 'suitcase', 'sunglasses',
            'sunscreen', 'swimsuit', 'swing set', 'tablet', 'tank', 'tape dispenser', 'taser', 'tea leaf', 'telescope',
            'television', 'tennis ball', 'tennis court', 'throw blanket', 'ticket machine', 'ticket stub', 'tire iron',
            'tissue box', 'toaster oven', 'tofu', 'toilet', 'toiletries bag', 'tomato', 'tongue depressor', 'tool belt',
            'tool box', 'toothbrush', 'toothbrush holder', 'toothpaste', 'touchscreen', 'tow truck', 'towel',
            'tracksuit', 'tractor shed', 'tractor', 'train brake', 'train door', 'trash bag', 'trash can', 'tricycle',
            'truck', 'tulip', 'turkey', 'turn signal', 'turnstiles', 'tweezers', 'udon noodles', 'ultraviolet lamp',
            'umbrella stand', 'underground river', 'unleavened bread', 'urinal', 'usb drives', 'utility box',
            'utility cart', 'vacuum cleaner', 'vanilla extract', 'vanity mirror', 'vans', 'vase', 'vegetable',
            'vegetation', 'veggie burger', 'velvet rope', 'vending machine', 'ventilation system', 'video camera',
            'violin bow', 'violin strings', 'virus', 'volcano', 'voucher', 'vr headset', 'walking stick', 'wall art',
            'wall decorations', 'wall street', 'wardrobe', 'washing machine', 'water', 'water bottle', 'water buffalo',
            'water fountain', 'water heater', 'water tank', 'watering can', 'webcam', 'wheelchair ramp', 'whistle',
            'willow tree', 'windbreak', 'wine aerator', 'wine decanter', 'wine opener', 'wine rack', 'wine tasting kit',
            'wire rack', 'wireless speaker', 'wrench', 'wrist weights', 'wristband', 'yoga block', 'yoga mat', 'yogurt',
            'zinc', 'afghanistan', 'albania', 'algeria', 'andorra', 'angola', 'antigua and barbuda', 'argentina',
            'armenia', 'australia', 'austria', 'azerbaijan', 'bahrain', 'bangladesh', 'barbados', 'belarus', 'belgium',
            'belize', 'benin', 'bhutan', 'bolivia', 'bosnia and herzegovina', 'botswana', 'brazil', 'brunei',
            'bulgaria', 'burkina faso', 'burundi', 'cambodia', 'cameroon', 'canada', 'cape verde',
            'central african republic', 'chad', 'chile', 'china', 'colombia', 'comoros', 'congo', 'costa rica',
            'croatia', 'cuba', 'cyprus', 'czech republic', 'democratic republic of the congo', 'denmark', 'djibouti',
            'dominica', 'dominican republic', 'ecuador', 'egypt', 'el salvador', 'england', 'equatorial guinea',
            'eritrea', 'estonia', 'ethiopia', 'federated states of micronesia', 'finland', 'france', 'gabon', 'gambia',
            'georgia', 'germany', 'ghana', 'greece', 'grenada', 'guatemala', 'guinea', 'guinea bissau', 'guyana',
            'haiti', 'honduras', 'hungary', 'iceland', 'india', 'indonesia', 'iran', 'iraq', 'ireland', 'israel',
            'italy', 'jamaica', 'japan', 'jordan', 'kazakhstan', 'kenya', 'kiribati', 'kosovo', 'kuwait', 'kyrgyzstan',
            'laos', 'latvia', 'lebanon', 'lesotho', 'liberia', 'libya', 'liechtenstein', 'lithuania', 'luxembourg',
            'madagascar', 'malawi', 'malaysia', 'maldives', 'mali', 'malta', 'marshall islands', 'mauritania', 'mauritius', 'mexico', 'moldova', 'monaco', 'mongolia', 'montenegro', 'morocco', 'mozambique', 'myanmar',
            'namibia', 'nauru', 'nepal', 'netherlands', 'new zealand', 'nicaragua', 'niger', 'nigeria', 'north korea',
            'norway', 'oman', 'pakistan', 'palau', 'palestine', 'panama', 'papua new guinea', 'paraguay', 'peru',
            'philippines', 'poland', 'portugal', 'qatar', 'romania', 'russia', 'rwanda', 'saint kitts and nevis',
            'saint lucia', 'saint vincent and the grenadines', 'samoa', 'san marino', 'sao tome and principe',
            'saudi arabia', 'senegal', 'serbia', 'seychelles', 'sierra leone', 'singapore', 'slovakia', 'slovenia',
            'solomon islands', 'somalia', 'south africa', 'south korea', 'spain', 'sudan', 'suriname', 'swaziland',
            'sweden', 'switzerland', 'syria', 'taiwan', 'tajikistan', 'tanzania', 'thailand', 'togo', 'tonga',
            'trinidad and tobago', 'tunisia', 'turkey', 'turkmenistan', 'tuvalu', 'uganda', 'ukraine',
            'united arab emirates', 'united kingdom', 'united states of america', 'uruguay', 'uzbekistan', 'vanuatu',
            'venezuela', 'vietnam', 'yemen', 'zambia', 'zimbabwe', 'amsterdam netherlands', 'anaheim california',
            'austin texas', 'auckland new zealand', 'asheville north carolina', 'ashgabat turkmenistan',
            'athens greece', 'athens georgia', 'atlanta georgia', 'antwerp belgium', 'adelaide australia',
            'astana kazakhstan', 'asuncion paraguay', 'algiers algeria', 'acapulco mexico', 'ankara turkey',
            'baghdad iraq', 'bangkok thailand', 'beijing china', 'berlin germany', 'boston massachusetts',
            'buenos aires argentina', 'bursa turkey', 'bucharest romania', 'baltimore maryland', 'beirut lebanon',
            'belfast northern ireland', 'bratislava slovakia', 'belgrade serbia', 'budapest hungary', 'baku azerbaijan',
            'bordeaux france', 'busan south korea', 'brussels belgium', 'bangalore india', 'calgary canada',
            'chicago illinois', 'copenhagen denmark', 'columbus ohio', 'cologne germany', 'cairo egypt',
            'cape town south africa', 'caracas venezuela', 'cleveland ohio', 'cork ireland', 'christchurch new zealand',
            'casablanca morocco', 'chengdu china', 'cannes france', 'canberra australia', 'dallas texas',
            'dubai united arab emirates', 'dhaka bangladesh', 'dakar senegal', 'delhi india', 'durban south africa',
            'dublin ireland', 'dalian china', 'doha qatar', 'denver colorado', 'dusseldorf germany',
            'davao city philippines', 'darwin australia', 'dunfermline scotland', 'daegu south korea', 'damascus syria',
            'dar es salaam tanzania', 'edinburgh scotland', 'edmonton canada', 'essen germany', 'evora portugal',
            'ensenada mexico', 'el paso texas', 'enugu nigeria', 'enschede netherlands', 'eureka california',
            'erie pennsylvania', 'eilat israel', 'essentuki russia', 'esbjerg denmark', 'fez morocco', 'florence italy',
            'frankfurt germany', 'fort worth texas', 'fukuoka japan', 'faisalabad pakistan',
            'fujairah united arab emirates', 'funafuti tuvalu', 'florianopolis brazil', 'flinders australia',
            'faro portugal', 'fujairah united arab emirates', 'fort mcmurray canada', 'fortaleza brazil',
            'friesland netherlands', 'funchal portugal', 'fuzhou china', 'fresno california', 'fermoy ireland',
            'fukushima japan', 'glasgow scotland', 'guangzhou china', 'gdansk poland', 'guatemala city guatemala',
            'guwahati india', 'gyeongju south korea', 'genoa italy', 'grahamstown south africa', 'guadalajara mexico',
            'geneva switzerland', 'graz austria', 'gwangju south korea', 'houston texas', 'hamburg germany',
            'hanoi vietnam', 'helsinki finland', 'ho chi minh city vietnam', 'haifa israel', 'havana cuba',
            'hong kong china', 'hobart australia', 'hangzhou china', 'hilo hawaii', 'hermosillo mexico',
            'honolulu hawaii', 'helsingborg sweden', 'hiroshima japan', 'harare zimbabwe', 'istanbul turkey',
            'indianapolis indiana', 'ibadan nigeria', 'istanbul turkey', 'indore india', 'izmir turkey',
            'isafahan iran', 'incheon south korea', 'innsbruck austria', 'islamabad pakistan', 'ingolstadt germany',
            'irvine california', 'irkutsk russia', 'jakarta indonesia', 'jerusalem israel', 'jacksonville florida',
            'johannesburg south africa', 'jabalpur india', 'jinan china', 'jeddah saudi arabia', 'jalapa guatemala',
            'jackson mississippi', 'juarez mexico', 'jabalpur india', 'jining china', 'kampala uganda',
            'kathmandu nepal', 'kaunas lithuania', 'kuala lumpur malaysia', 'kyoto japan', 'kagoshima japan',
            'karachi pakistan', 'kiev ukraine', 'kingston jamaica', 'kolkata india', 'kunming china',
            'kabul afghanistan', 'kyiv ukraine', 'kawasaki japan', 'london england', 'la paz bolivia',
            'los angeles california', 'lima peru', 'lyon france', 'lisbon portugal', 'luanda angola',
            'liverpool england', 'lagos nigeria', 'leeds england', 'ljubljana slovenia', 'lyon france', 'lima peru',
            'lviv ukraine', 'leipzig germany', 'lusaka zambia', 'lausanne switzerland', 'madrid spain',
            'manchester england', 'mexico city mexico', 'manila philippines', 'montreal canada', 'milan italy',
            'moscow russia', 'madrid spain', 'mumbai india', 'managua nicaragua', 'melbourne australia',
            'marrakech morocco', 'miami florida', 'minneapolis minnesota', 'mecca saudi arabia', 'melbourne australia',
            'makati philippines', 'monterrey mexico', 'nagoya japan', 'new york city', 'nanjing china',
            'new delhi india', 'nantes france', 'noida india', 'newcastle upon tyne england', 'nice france',
            'nurumberg germany', 'new orleans louisiana', 'nairobi kenya', 'naples italy', 'noosa australia',
            'osaka japan', 'oklahoma city oklahoma', 'oslo norway', 'oxford england', 'ottawa canada', 'orsay france',
            'odessa ukraine', 'oranjestad aruba', 'orlando florida', 'ostrava czech republic', 'oaxaca mexico',
            'otago new zealand', 'ouagadougou burkina faso', 'odense denmark', 'oulu finland', 'paris france',
            'prague czech republic', 'porto portugal', 'philadelphia pennsylvania', 'pyeongyang north korea',
            'perth australia', 'plovdiv bulgaria', 'pattaya thailand', 'portland oregon', 'phoenix arizona',
            'porto alegre brazil', 'peshawar pakistan', 'panama city panama', 'rome italy', 'rio de janeiro brazil',
            'riyadh saudi arabia', 'reykjavik iceland', 'rotterdam netherlands', 'ras al khaimah united arab emirates',
            'raleigh north carolina', 'riga latvia', 'rochester new york', 'recife brazil', 'san francisco california',
            'sydney australia', 'singapore', 'seoul south korea', 'stockholm sweden', 'santiago chile',
            'san diego california', 'shanghai china', 'sao paulo brazil', 'stuttgart germany', 'sevilla spain',
            'saskatoon canada', 'san salvador el salvador', 'sofia bulgaria', 'seattle washington', 'tokyo japan',
            'torino italy', 'tunis tunisia', 'tashkent uzbekistan', 'toronto canada', 'tirana albania',
            'tijuana mexico', 'turin italy', 'tokyo japan', 'thessaloniki greece', 'taegu south korea', 'taksim turkey',
            'taipei taiwan', 'tripoli libya', 'tokyo japan', 'ulaanbaatar mongolia', 'ubud indonesia', 'uppsala sweden',
            'urumqi china', 'vaduz liechtenstein', 'vancouver canada', 'valencia spain', 'vigo spain','valparaiso chile', 
            'vladivostok russia', 'vienna austria', 'vilnius lithuania', 'villarreal spain',
            'washington dc', 'westminster england', 'wilmington delaware', 'wroclaw poland', 'warsaw poland',
            'wellington new zealand', 'winnipeg manitoba', 'warsaw poland', 'wuhan china', 'yokohama japan',
            'york england', 'yaounde cameroon', 'yuma arizona', 'ypres belgium', 'yakutsk russia', 'yerevan armenia',
            'yanbu saudi arabia', 'yogyakarta indonesia', 'yekaterinburg russia', 'zacatecas mexico', 'zunyi china',
            'zincantan mexico', 'zagreb croatia', 'zeeland netherlands', 'zhongshan china', 'zanzibar tanzania',
            'zurich switzerland', 'zaragoza spain', 'denali', 'mount saint lias', 'mount whitney', 'mount rainier',
            'iztaccihuatl', 'grand teton', 'gannett peak', 'mount adams', 'mount saint helens', 'mount shasta',
            'mount saint helens', 'pikes peak', 'aconcagua', 'fitz roy', 'cotopaxi', 'chimborazo', 'mont blanc',
            'zugspitze', 'mount elbrus', 'mount etna', 'everest', 'k2', 'lhotse', 'makalu', 'cho oyu', 'manaslu',
            'annapurna', 'dhaulagiri', 'nanga parbat', 'kangchenjunga', 'mount fuji', 'kilimanjaro', 'meru', 'aoraki',
            'haleakala', 'puncak jaya', 'sumantri', 'amazon', 'colorado river', 'dnieper', 'ganges', 'illinois river',
            'mississippi river', 'nile', 'rhine', 'yangtze river', 'yellow river', 'zambezi river', 'yenisei river']  # さまざまな単語や地名、山川などが含まれるリストを作成する

In [ ]:
keywords = sorted(allwords)  # allwordsリストの単語をアルファベット順にソートして、keywordsという新しいリストを作成する。これにより、単語の一覧を見やすく整理することができる。

In [ ]:
def extract_testword(question):
    # テストワードは、ダブルクォートで囲まれた文字列であると単純に定義する
    match = re.search(r'"(.+)"', question)  # 正規表現を使ってダブルクォート内の文字列を検索
    if match:
        testword = match.group(1)  # マッチした場合、その文字列を取得
        # print(f'The testword is: {testword}')  # テストワードを出力（コメントアウト）
    else:
        print(f'Testword not found in question: {question}')  # テストワードが見つからなかった場合のメッセージ
        testword = ''

    return testword  # テストワードを返す


def agent_beta(obs, cfg):
    # agent_beta関数は、観察(observation)と設定(configuration)を受け取り、エージェントの応答を生成する

    if obs.turnType == 'ask':  # 'ask'ターンの場合
        response = agent_alpha(obs, cfg)  # agent_alpha関数を呼び出し応答を得る

    elif obs.turnType == 'guess':  # 'guess'ターンの場合
        if random.random() < 0.5:  # 確率的に応答を選ぶ
            # かわいいマーケティングの応答を選ぶかもしれません
            responses = ['no alpha, no fun', 'you better check out alpha, mister', 'alpha is life',
                         'alphabetical bisection can help you', f'creamy & milky, alpha {VERSION}',
                         'make kaggle great again']

            response = random.choice(responses)  # ランダムに応答を選択
        else:
            # 次の場合はagent_alphaを使う
            response = agent_alpha(obs, cfg)

    else:  # 'answer'ターンの場合
        assert obs.turnType == 'answer'  # ここではanswerターンであることをアサート
        response = 'no'  # 標準応答は'no'

    return response  # 応答を返す


def agent_alpha(obs, cfg):
    global keywords  # グローバル変数keywordsを使用

    if obs.turnType == 'ask':  # 'ask'ターンの場合

        if keywords:  # keywordsリストが存在する場合
            # 質問のバリエーションを交互に行う。文法は重要ではないかもしれないが賢いLLMは正しく応答できるかも
            testword = keywords[len(keywords) // 2]  # keywordsリストの真ん中の単語を取得
            responses = [
                f'Does the keyword (in lowercase) precede "{testword}" in lexicographical order?',
                f'Does the keyword (in lowercase) come before "{testword}" in alphabetical order?',
                f'Does the keyword (in lowercase) come before "{testword}" in sorting order?',
                f'Does the keyword (in lowercase) precede "{testword}" in alphabetical order?',
            ]
            response = responses[len(obs.questions) % len(responses)]  # 質問数に基づいて応答を選択
        else:
            response = f'space exhausted.. restarting, version {VERSION}'  # keywordsが空になった場合のメッセージ
            # リスタート
            keywords = sorted(allwords)  # allwordsを再ソートしてkeywordsを更新する

    elif obs.turnType == 'guess':  # 'guess'ターンの場合

        assert len(obs.answers) > 1  # あなたの回答が1つより多いことを確認

        # 最後の質問と回答に基づいて空間を二分探索
        testword = extract_testword(obs.questions[-1])  # 最後の質問からテストワードを抽出
        if testword:
            if obs.answers[-1] == 'yes':
                keywords = [k for k in keywords if k < testword]  # 'yes'のときテストワードより前のものを残す
            else:
                keywords = [k for k in keywords if not k < testword]  # 'no'のときテストワードより後のものを残す

        if keywords:  # keywordsが存在する場合
            guess = keywords[0]  # 最初のkeywordを推測として選択
            keywords = keywords[1:]  # 使用したkeywordはリストから削除
            response = guess  # 推測を応答として返す

        else:  # keywordsが空になった場合
            # すべての候補を消費した可能性がある
            response = 'no candidates left'  # 候補が残っていないことを示す応答

    else:  # 'answer'ターンの場合
        assert obs.turnType == 'answer'  # ここではanswerターンであることをアサート

        # 最初の質問の応答は'yes'にして、Alphaの質問者が進められるようにする
        if len(obs.questions) == 1:
            response = 'yes'  # 最初の質問には'yes'と返す

        else:
            # Alphaをプレイする必要があるが、テストワードを質問から見つけることができるか？
            testword = extract_testword(obs.questions[-1])  # 最後の質問からテストワードを取得
            if testword:
                # keywordとテストワードを比較し、正規化して小文字にする！
                response = 'yes' if obs.keyword.lower() < testword else 'no'  # テストワードより小さいかの判定を返す
            else:
                # テストワードが見つからなかった場合、Alphaをプレイしていないかもしれない
                response = 'no'  # その場合は'no'と返す

    return response  # 最終的な応答を返す

In [ ]:
def switchboard(obs, cfg):
    # スイッチボード関数は、観察(observation)と設定(configuration)を受け取り、処理の流れを管理する

    if obs.turnType == 'ask':  # 'ask'ターンの場合

        if len(obs.questions) == 0:  # 初めての質問の場合
            # LLMが'no'と答えるけれども、Alphaが認識できる質問を使用する
            response = 'Is it Agent Alpha?'  # 初めの質問として設定する

        else:
            if obs.answers[0] == 'yes':  # 最初の回答が'yes'の場合
                response = agent_alpha(obs, cfg)  # agent_alphaを使用
            else:
                response = agent_beta(obs, cfg)  # agent_betaを使用

    elif obs.turnType == 'guess':  # 'guess'ターンの場合

        assert len(obs.questions) > 0 and len(obs.answers) > 0  # 質問と回答があることを確認

        if len(obs.answers) == 1:  # 回答が1つだけの場合
            if obs.answers[-1] == 'yes':  # 最後の回答が'yes'の場合
                response = f'bingo! version {VERSION}'  # 正解したのでメッセージを返す
            else:
                response = f'too bad..'  # 不正解の場合のメッセージ
                
                # ここでagent_betaを呼び出すことも可能
                # response = agent_beta(obs, cfg)

        else:
            # 最初の質問が'yes'であればAlphaをプレイする
            play_alpha = obs.answers[0] == 'yes'  # 最初の回答が'yes'の場合はTrue

            if play_alpha:
                response = agent_alpha(obs, cfg)  # agent_alphaを使用
            else:
                response = agent_beta(obs, cfg)  # agent_betaを使用

    elif obs.turnType == 'answer':  # 'answer'ターンの場合

        assert len(obs.questions) > 0  # 質問があることを確認
        questioner_is_alpha = 'agent alpha' in obs.questions[0].lower()  # 質問者がAlphaか確認

        if questioner_is_alpha:  # 質問者がAlphaの場合
            response = agent_alpha(obs, cfg)  # agent_alphaを使用

        else:
            # 質問者がAlphaでないので、LLMを使う
            response = agent_beta(obs, cfg)  # agent_betaを使用

        assert response in ['yes', 'no']  # 応答が'yes'か'no'であることを確認

    else:
        # ここには到達しないことが期待される
        assert False, f'Unexpected turnType: {obs.turnType}'  # 異常なturnTypeの場合はエラーを投げる

    return response  # 最終的な応答を返す


def agent_fn(obs, cfg):
    """ メインのフック、'agent_fn'という名前を維持。 """

    try:
        response = switchboard(obs, cfg)  # switchboard関数を呼び出して応答を得る
    except AssertionError:
        import traceback
        traceback.print_exc()  # エラーが発生した場合はトレースバックを表示
        response = 'no'  # エラーが発生した場合のデフォルト応答

    return response  # 応答を返す

In [ ]:
from kaggle_environments import make  # Kaggleの環境を作成するためのライブラリをインポート

def agent_dummy(obs, cfg):
    # ダミーエージェント（単純なエージェント）を定義
    if obs.turnType == "ask":  # 'ask'ターンの場合
        response = "Is it a duck?"  # 質問に対する応答
    elif obs.turnType == "guess":  # 'guess'ターンの場合
        response = "duck"  # 推測に対する応答
    elif obs.turnType == "answer":  # 'answer'ターンの場合
        response = "no"  # 答えに対する応答
    return response  # 応答を返す


# デバッグ用の設定を指定
debug_config = {
    'episodeSteps': 61,     # 初期ステップおよびラウンドごとの3ステップ（質問/回答/推測）の合計
    'actTimeout': 60,       # ラウンドごとのエージェントの制限時間（秒）
    'runTimeout': 1200,     # エピソードの最大実行時間（秒）
    'agentTimeout': 3600    # 非推奨フィールド（エージェントの最大実行時間）
}

# 環境を作成
env = make(environment="llm_20_questions", configuration=debug_config, debug=True)

# ゲームを実行。2つのエージェントはagent_fn、2つのエージェントはagent_dummyを使用。
game_output = env.run(agents=[agent_fn, agent_fn, agent_dummy, agent_dummy])

# 結果を表示する。
env.render(mode="ipython", width=1080, height=700)  # Jupyterノートブック内で環境を表示